Bokeh's drawing tools are the basis for a wide range of functionality in EarthSim, using the convenient interface provided by [HoloViews](http://holoviews.org).  They make it simple to build systems for annotating existing data, highlighting regions of interest, and drawing and editing shapes that can be used as input to simulators or other programs. This user guide will give a basic introduction to the drawing tools, including how to access the resulting data from within Python code.

For more detail about the underlying Bokeh tools, see the [Bokeh user guide](https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#userguide-tools-edit). Note that most of the discussion here is not specific to EarthSim, and applies to any usage of the drawing tools in practice, apart from a few I/O routines imported from `earthsim` when used below.

<style>.container { width:100% !important; }</style>

In [ ]:
import os
import numpy as np
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs

from holoviews.streams import PointDraw, PolyEdit, BoxEdit, PolyDraw

url = 'http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png'
tiles = gv.WMTS(url, extents=(-91, 32.2, -90.8, 32.4), crs=ccrs.PlateCarree())

hv.extension('bokeh')

## Drawing Points

All drawing tools are added by instantiating a corresponding [HoloViews stream](http://holoviews.org/user_guide/Responding_to_Events.html), which also syncs the data. Here we will use the ``PointDraw`` stream, which allows adding points, dragging points, and deleting points.

**Add point**: Tap anywhere on the plot; each tap adds one point.

**Move point**: Tap and drag an existing point, which will be dropped once you let go of the mouse button.

**Delete point**: Tap a point to select it, then press the Backspace key (sometimes labeled "Delete") while the mouse is within the plot area.

Note that to use the `PointDraw` tool or any of the other drawing tools, you first need to select the icon for it in the toolbar:<img src="https://bokeh.pydata.org/en/latest/_images/PointDraw.png">

In [ ]:
%%opts Points [width=900 height=500 tools=['hover']] (size=10 color='red')
points = gv.Points(np.random.rand(10, 2)*2000000, crs=ccrs.GOOGLE_MERCATOR)
point_stream = PointDraw(source=points)
tiles * points

Note that here and in the other examples below, we have provided initial values for the `source`, just so that there will be objects in the map when this notebook is rendered as a web page or otherwise shared. In practice, the `source` here and in every case below can be an empty list `[]` if you don't want any initial values.

Once points are available on the map, we can wrap them in a GeoViews Points object, project them back into longitude and latitude, and then convert the resulting object to a dataframe for use in any Python code:

In [ ]:
if point_stream.data:
    projected = gv.operation.project_points(point_stream.element, projection=ccrs.PlateCarree())
    display(projected.dframe())

Of course, the dataframe output above will only contain the points that were present at the time that cell was executed, so the cell will need to be re-run if you add points to the main plot.

## Drawing bounding boxes

The ``BoxEdit`` stream adds a tool that allows drawing, dragging, and deleting rectangular bounding boxes, once you have selected it in the toolbar: <img src="https://bokeh.pydata.org/en/latest/_images/BoxEdit.png">

**Add box**: Hold shift, then click and drag anywhere on the plot.

**Move box**: Click and drag an existing box; the box will be dropped once you let go of the mouse button.

**Delete box**: Tap a box to select it, then press the Backspace (or Delete) key while the mouse is within the plot area.

In [ ]:
%%opts Polygons [width=900 height=500] 
%%opts Polygons (fill_alpha=0 line_color='black' selection_fill_color='red')

sample_box = [[-10129181., 3796743.], [-10129181., 3812041.], [-10113192., 3812041.], 
              [-10113192., 3796743.], [-10129181., 3796743.]]

box_poly = gv.Polygons([sample_box], crs=ccrs.GOOGLE_MERCATOR)
box_stream = BoxEdit(source=box_poly)
tiles * box_poly

Note that `BoxEdit` accepts a `Polygon` element, as there is not yet a vectorized Box type that would let it generate boxes directly, and so we will need to convert the returned polygons into boxes manually:

In [ ]:
def bbox(poly):
    "Convert the polygon returned by the BoxEdit stream into a bounding box tuple"
    xs,ys = poly.array().T
    return (xs[0], ys[0], xs[2], ys[2])

if box_stream.element:
    polygons = gv.operation.project_path(box_stream.element, 
                                         projection=ccrs.PlateCarree()).split()
    bboxes = [bbox(p) for p in polygons]
    print(bboxes)

(Of course, boxes will only be printed above if they were drawn on the map before the cell above is executed.)

## Polygon Editing

The ``PolyEdit`` stream adds a Bokeh tool to the source plot that allows drawing, dragging, and deleting vertices on polygons and making the drawn data available to Python:<img src="https://bokeh.pydata.org/en/latest/_images/PolyEdit.png">

The tool supports the following actions:

**Show vertices**: Double tap an existing patch or multi-line

**Add vertex**: Double tap an existing vertex to select it, then the tool will draw the next point; to add it tap in a new location. To finish editing and add a point, double tap; otherwise press the ESC key to cancel.

**Move vertex**: Drag an existing vertex and let go of the mouse button to release it.

**Delete vertex**: After selecting one or more vertices press Backspace (or Delete) while the mouse cursor is within the plot area.

In [ ]:
%%opts Shape [width=900 height=500 tools=['box_select']] (alpha=0.5)

shapefile = '../data/vicksburg_watershed/watershed_boundary.shp'
mask_shape = gv.Shape.from_shapefile(shapefile)[0]
vertex_stream = PolyEdit(source=mask_shape)
tiles * mask_shape

Once the shape has been edited, it can be pulled out into its own file for later usage, and displayed separately:

In [ ]:
%%opts Shape [width=600 height=400] (alpha=0.5)
from earthsim.io import save_shapefile
if vertex_stream.data:
    edited_shape_fname = '../data/vicksburg_watershed_edited/watershed_boundary.shp'
    dir_name = os.path.dirname(edited_shape_fname)
    if not os.path.isdir(dir_name): os.makedirs(dir_name)
    save_shapefile(vertex_stream.data, edited_shape_fname, shapefile)
    mask_shape = gv.Shape.from_shapefile(edited_shape_fname)
mask_shape = mask_shape.opts() # Clear options to avoid adding edit tool
mask_shape.last.clone(crs=ccrs.GOOGLE_MERCATOR)

## Drawing Polygons

The ``PolyDraw`` tool allows drawing new polygons or paths (polylines) on a plot, depending on whether it is given a Path or Polygon source:<img src="https://bokeh.pydata.org/en/latest/_images/PolyDraw.png">

**Add patch/multi-line**: Double tap to add the first vertex, then use tap to add each subsequent vertex. To finalize the draw action, double tap to insert the final vertex or press the ESC key to stop drawing.

**Move patch/multi-line**: Tap and drag an existing patch/polyline; the point will be dropped once you let go of the mouse button.

**Delete patch/multi-line**: Tap a patch/multi-line to select it, then press Backspace/Delete while the mouse is within the plot area.

In [ ]:
%%opts Polygons [width=900 height=500] (fill_alpha=0.1 line_color='black') 
%%opts Path (line_width=5 color='black')

sample_poly=dict(
    Longitude = [-10114986, -10123906, -10130333, -10121522, -10129889, -10122959],
    Latitude  = [  3806790,   3812413,   3807530,   3805407,   3798394,   3796693])
sample_path=dict(
    Longitude = [-10129501, -10119319, -10119114, -10127543],
    Latitude  = [  3809682,   3807677,   3805097,   3796477])

new_polys = gv.Polygons([sample_poly], crs=ccrs.GOOGLE_MERCATOR)
new_paths = gv.Path([sample_path], crs=ccrs.GOOGLE_MERCATOR)
poly_stream = PolyDraw(source=new_polys)
path_stream = PolyDraw(source=new_paths)
tiles * new_polys * new_paths

In [ ]:
path_stream.element.data

Notice that the toolbar has two `PolyDraw` tools here; if you select the first one you'll be able to add `Polygons` (drawn with thin lines), and if you select the other one you can add `Path` objects (poly-lines, drawn with a thick line).  You'll need to have the appropriate copy of the tool selected if you want to move or delete an object associated with that stream.

Once you have drawn some objects, you can extract the new paths or polygons from the stream (which will be blank unless you have drawn something above when the following cells are executed):

In [ ]:
poly_stream.element.geom()

In [ ]:
path_stream.element.geom()

Here `.geom()` returns a [Shapely geometry](https://toblerity.org/shapely/shapely.geometry.html) with all of the shapes you drew of that type.  If you would rather work with each shape separately, you can get them as a list with `poly_stream.element.split()` or `path_stream.element.split()`.

## Drawing and editing a polygon

By adding tools for both polygon drawing and vertex editing on the same HoloViews object, we can both draw and edit polygons in the same plot:

In [ ]:
%%opts Polygons [width=900 height=500] (fill_alpha=0.2 line_color='black')
new_polys = gv.Polygons([sample_poly], crs=ccrs.GOOGLE_MERCATOR)
poly_stream   = PolyDraw(source=new_polys)
vertex_stream = PolyEdit(source=new_polys)
tiles * new_polys

In [ ]:
poly_stream.data

In [ ]:
poly_stream.element

The above examples should make it clear how to draw shapes and use the data from within Python.  The next set of examples show how to associate data interactively with each point or object added, via [Annotators](Annotators.ipynb).